In [3]:
import multiprocessing
import os
import tqdm
import re
import time

def download_xmls(number):   
    download_command = f'wget https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/pubmed23n{number}.xml.gz'
    cd_command = 'cd xml_data'
    
    os.system(f'{cd_command} && {download_command}')
    
def obtain_existing_files():
    pattern = r'\d{4}'
    files = os.listdir('xml_data/')
    existing = []
    for file in files:
        existing.append(re.search(pattern, file).group(0))
    
    return existing
    
def download_all_files(time_out):
    while True:
        indices_to_download = []
        existing_indices = obtain_existing_files()
        arg_lst = []
        print(f"{len(existing_indices)} files exist")

        if len(existing_indices) == 1166:
            break


        for i in range(1,1167):
            str_idx = str(i)
            str_idx = '0' * (4-len(str_idx)) + str_idx

            if str_idx in existing_indices:
                continue

            arg_lst.append((str_idx,))

        pool = multiprocessing.Pool(processes=128)
        pool.starmap(download_xmls, arg_lst)

        pool.join()
        
        time.sleep(time_out)


#download_all_files(60)
    
# lst = [(i,) for i in range(1, 1166+1)]
# for i in tqdm.tqdm(range(1, 1167)):
#     download_xmls(i)


# lst = [(i,) for i in range(1166+1)]
# pool = multiprocessing.Pool(processes=128)
# pool.starmap(download_xmls, lst)
# obtain_existing_files()
# print(f'number of downloaded file is {len(os.listdir("xml_data/"))}')

In [4]:
import json
import numpy as np
stat = {}


# Open and read the JSON file
with open('output.json', 'r') as json_file:
    # Load the JSON content into a dictionary
    output = json.load(json_file)

print(f"total files processed {len(output)}")
for key, value in output.items():
    for subkey in value:
        if not subkey in stat:
            stat[subkey] = [value[subkey]]
        else:
            stat[subkey].append(value[subkey])

averaged = {}
for key, value in stat.items():
    if key == 'PubmedArticle':
        averaged[key] = np.sum(value)
        print(np.mean(averaged[key]))

for key, value in stat.items():
    if key == 'PubmedArticle':
        continue
    
    else:
        averaged[key] = np.sum(value)/averaged['PubmedArticle']
        
for key, value in stat.items():
    if key == 'PubmedArticle':
        averaged[key] = averaged[key] // 30000
    
print(averaged)

total files processed 1167


TypeError: 'float' object is not iterable

In [17]:
xml_files = os.listdir('/shared/pubmed/')
print(len(xml_files))

for file in xml_files:
    if '.gz' in file:
        xml_file_name = file[:-3]
        command = f'gunzip {file}'
        os.system(command)
        print(command)
        
        command = f'mv {xml_file_name} /shared/xml_data/'
        os.system(command)
        print(command)
        break
        

1538
gunzip pubmed23n0223.xml.gz
mv pubmed23n0223.xml /shared/xml_data/
